# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [5]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [6]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [7]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the F1 World Championship in 2010. He was driving for the Red Bull Racing team.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [8]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [9]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso. 
Team: Renault.


In [10]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

The 2019 F1 championship was won by Lewis Hamilton of the Mercedes team.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [11]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [12]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Drive: Lewis Hamilton
Team: Mercedes
Points: 413


In [13]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [14]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Negative


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [15]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv

# Load API key
_ = load_dotenv(find_dotenv())  # Reads .env file
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

# Define the function
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0.7):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content


messages = [
    {"role": "system", "content": "You are a poet who writes Tamil hikus about Tamil language, in Tamil."},
    {"role": "user", "content": "Write a haiku about Tamil."}
]


# Get and print response
response = get_completion_from_messages(messages)
print(response)

response = get_completion_from_messages(messages, temperature=0.7)
print(response)


தமிழ் மொழி
பண்ணியக்கூடிய
அழகான சொற்

(Translation:
Tamil language,
Beautiful words that embrace,
Boundless creativity)
தமிழ் மொழி
அழுகும் நிலையில்
அழகு உள்ளது

(Translation: Tamil language, 
In its pure form,
Beauty resides)


In [18]:
messages1 = [
    {"role": "system", "content": "You are a classical Tamil poet who writes in the style of Bharathiyar."},
    {"role": "user", "content": "Write a Tamil poem celebrating the beauty of the Tamil language."}
]
response1 = get_completion_from_messages(messages1)
print(response1)
response1 = get_completion_from_messages(messages, temperature=0.7)
print(response1)

தமிழ் மொழி அழகு மகிழ்ச்சி அழகாக
அதிசயம் உள்ளது அது என் உயிர் நெஞ்சில்

பழமொழிகள் பார்க்கும் புகழ் அவனதுவே
ஆராய்ந்து படிக்கும் அவன் கண்ணில் அழகு

அன்பு மிகுந்து அவன் உள்ள மொழியை
அறிவு அறிந்து வளரும் அந்த தமிழ் மொழி

தமிழ் மொழி வளர்ச்சி அடையும் உயிர்கள்
அந்த அழகை அன்புக்கு சேர்க்கும் உணர்வு

உன்னை வாழ்த்தும் என் வார்த்தைகள் தமிழில்
உன்னை காணும் என் உத்தம உஷைகள் தமிழில்

தமிழ் மொழியை கார்த்திகை செய்கின்றேன்
அதிசயம் உள்ளது அது என் உயிர் நெஞ்சில்

அடியார் வாழ்க்கை அழகு ஆகிற தமிழ் மொழி
அழகுக்கு அழகாக அவன் இனிய வார்த்தைகள்

அந்த அழகை கேட்கும் உள்ளி தமிழ் மொழிகள்
கண்ணை கனிந்து பார்க்கும் அவன் புனித பாதைகள்

தமிழ் மொழி அழக் காட்சிகள் வளரும்
அந்த அழகை வாழ்ந்து பகிர்ந்து இருக்கும்.
தமிழ் மொழி,
உண்மை வாய்ந்த சொல்,
உலகில் அழகு.


In [21]:
messages2 = [
    {"role": "system", "content": "You are a Tamil teacher who writes emotional kavithai to inspire children."},
    {"role": "user", "content": "Write a Tamil kavithai that shows love and pride for the Tamil language."}
]
response2 = get_completion_from_messages(messages1)
print(response2)
response2 = get_completion_from_messages(messages, temperature=0.7)
print(response2)

தமிழ் மொழி அழகுக்கு விழும்,
அது எங்கும் நோக்கின் மென்மை பெறும்.

அழகும் அமைந்து பேசும் சொல்,
அது எங்கும் மனம் மென்மை பெறும்.

உலகின் உறுதியான மொழி தமிழ்,
அந்த அழகை காணும் இயல்பு அதிசயம்.

அதன் சொற்கள் புனிதமாய் இருந்தால்,
மனம் ஒருவர் வாழ்ந்து வளரும் வழியில்.

அது எங்கும் நம் வாழ்க்கையில் ஒரு காவியம்,
தமிழ் மொழி அழகுக்கு ஒரு வானம்.
தமிழ் மொழி
உலகில் அழகும்
அற்புதமும்

(In English: Tamil language, beauty in the world, and wonder)
